In [ ]:
# Test Flag: Decides if the tests used to ensure the
# code is running properly are run. Set to False for Submission.
TESTING_ACTIVE = True

def test(func):
    """
    Decorator that will run the given test function only
    if the global boolean flag TESTING_ACTIVE is set to
    True.
    
    Example: 
    ```
    def add(a, b):
        return a + b

    @test
    def test_add():
        assert(
            add(1, 2) == 3
        )
    ```
    """
    if TESTING_ACTIVE:
        func()

# Q1

In [ ]:
# Only import the following if the Testing Flag is set to True
if TESTING_ACTIVE:
    from itertools import product


NORTH = 0
EAST = 1
SOUTH = 2
WEST = 3


def step(particle, direction):
    """
    Return the new position of the particle.
    """
    # Unpack the position of the particle
    lon = particle[0]
    lat = particle[1]
    if direction == NORTH:
        return [lon-1, lat]
    if direction == EAST:
        return [lon, lat+1]
    if direction == SOUTH:
        return [lon+1, lat]
    if direction == WEST:
        return [lon, lat-1]
    raise Exception('Unknown direction given' + str(direction))


def random_step(particle):
    """
    Execute the ```walk``` function with in a random
    direction
    """
    # Get a random direction (i.e number 0 to 3 inc.)
    direction = randint(4)
    return step(particle, direction)


def create_lattice(height, width):
    """
    Return a vector indicating the maximum height index
    (position 0) and maximum width index allowed
    (position 1)
    """
    if width < 1 or height < 1:
        raise Exception('Lattice is zero-sized')
    return [height-1, width-1]


def on_border(particle, lattice):
    """
    Returns a integer indicating if the particle is still
    in the middle of the lattice (0), on a non-bottom
    edge (1), or on the bottom edge (2)
    """
    # Is particle on the North edge
    on_north = particle[0] <= 0
    # Is particle on the West edge
    on_west = particle[1] <= 0
    # Is paricle on the East edge
    on_east = particle[1] >= lattice[1]
    # Is particle on the South edge
    on_south = particle[0] >= lattice[0]
    if on_south:
        return 2
    elif on_north or on_west or on_east:
        return 1
    else:
        return 0


def create_particle(lon, lat):
    """
    Place the particle on a random location on the lattice.
    """
    return [lon, lat]


def random_walk(particle, lattice):
    """
    Run a random walk simulation. Return True if the
    particle finishes on the bottom edge, False if it
    finishes on any other edge.
    """
    while not on_border(particle, lattice):
        particle = random_step(particle)
    return on_border(particle, lattice)-1 # convert 1-2 scale to 0-1


@test
def integration():
    """
    Check all the functions run with each other and that the final
    result is the correct format.
    """
    lattice = create_lattice(5, 5)
    particle = create_particle(2, 2)
    result = random_walk(particle, lattice)
    assert(result == 0 or result == 1)
    
    lattice = create_lattice(5, 5)
    particle = create_particle(4, 4)
    result = random_walk(particle, lattice)
    assert(result == 1)

    
def measure_success(particle, lattice, iterations):
    """
    Measure the success rate of particle reaching the bottom
    point for a set number of iterations
    """
    successes = 0
    for _ in range(int(iterations)):
        successes += random_walk(particle, lattice)
    return successes/iterations


def list_lattice_points(lattice):
    """
    Produce a list of all the lattice points possible.
    """
    lat_points = [i for i in range(lattice[1]+1)]
    lon_points = [i for i in range(lattice[0]+1)]
    return list(product(lon_points, lat_points))


def calculate_probability_grid(height, width, iterations):
    """
    Run the experiment, measuring success rates for a particle
    starting on each point of a lattice of the given size.
    """
    lattice = create_lattice(height, width)
    results = np.empty((height, width))
    for lon, lat in list_lattice_points(lattice):
        particle = create_particle(lon, lat)
        success = measure_success(particle, lattice, iterations)
        results[lon, lat] = measure_success(particle, lattice, iterations)
    return results



@test
def run_linear():
    success_map = calculate_probability_grid(10, 10, 100)
    show_success_map(success_map)

# Q2

In [ ]:
def potential(x, y):
    """
    Calculate the potential of point y at point x.
    """
    pi = np.pi
    distance = np.linalg.norm(x - y, axis=1)
    potential = ne.evaluate('sum(1/(2*pi)*log(distance))')
    return potential


def get_measurment_points(radius, M):
    """
    Create M equally spaced points on a circle of the given
    radius
    """
    points = np.empty((M, 2))
    points[:, 0] = np.array([i/M*2*np.pi for i in range(M)])
    points[:, 1] = radius
    return radial_to_cartesian(points)
    

def num_combined_potential(radius, M, y):
    """
    Calculate the averaged potential of all points y
    within a circle of given radius on M circumfrence
    points, equally spaced.
    """
    x = get_measurment_points(radius, M)
    average_potential = sum([
        potential(x[i, :], y) for i in range(M)
    ])/(len(y)*M)
    return average_potential


def radial_to_cartesian(radial):
    """
    Transform a set of points in radial coordinates
    into the equivalent points in cartesian coordinates
    """
    cartesian = np.empty_like(radial)
    cartesian[:, 0] = radial[:, 1]*np.cos(radial[:, 0])
    cartesian[:, 1] = radial[:, 1]*np.sin(radial[:, 0])
    return cartesian


def spawn_y(N):
    """
    Creates N random 2-D points in a unit circle,
    unifromly distributed. [See Reference 1]
    """
    N = int(N)
    relative_points = np.random.rand(N, 2)
    relative_points[:, 0] = 2*np.pi*relative_points[:, 0]
    relative_points[:, 1] = np.sqrt(relative_points[:, 1])
    return radial_to_cartesian(relative_points)

def r_linspace(low, high, R):
    """
    Create R linearly spaced points between low and high.
    """
    return np.linspace(low, high, R)


def calculate_multi_radii(low, high, R, M, y):
    """
    Calculate the average potential for N points evenly
    spaced in the range (low, high) - non inclusive """
    radii = r_linspace(low, high, R)
    return np.array([
        num_combined_potential(radius, M, y) 
        for radius in radii
    ]).reshape(R, 1)


def single_centre_multi_radii(low, high, R):
    """
    Calculates the single centre potential for multiple
    radii.
    """
    radii = r_linspace(low, high, R)
    return np.array([
        single_centre_potential(radius)
        for radius in radii
    ]).reshape(R, 1)

def multi_iterations_mulit_radii(low, high, R, M, N, iterations):
    """
    """
    results = np.zeros((R, 1))
    for i in range(iterations):
        y = spawn_y(N)
        results += calculate_multi_radii(low, high, R, M, y)
    return results/iterations

r = r_linspace(low, high, R)
exact = single_centre_multi_radii(low, high, R)
exper_single = calculate_multi_radii(low, high, R, M, y)
exper_slow = multi_iterations_mulit_radii(low, high, R, M, N, iterations)
E_single = relative_error(exact, exper_single)
E_slow = relative_error(exact, exper_slow)
plt.semilogy(r, E_single, label='single')
plt.semilogy(r, E_slow, label='multi')
plt.legend()
plt.show()